# ML localization LTE Cat-M: XGBoost and Random Forest

## Data preparation

In [38]:
import pandas as pd
import numpy as np


In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
nr_df = pd.read_csv('/content/drive/MyDrive/localization/LTEM_fingerprint_DB_grid.csv', sep=',')
nr_df

,Timestamp,Lat,Lon,BMAXMM1,BMAXMM2,BMAXMM3,BMBACM1,BMBACM2,BMBAKN1,BMBAKN2,...,BMZVON2,BMZVON3,TBTS1M1,TBTS3M1,TBTS3M2,TBTS5N1,TBTS5N2,TBTS5N3,TBTS6M1,TBTS6M2
0,1701853011,49.227137,16.575807,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1701853012,49.227134,16.575806,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1701853013,49.227133,16.575806,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1701853014,49.227133,16.575805,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1701853015,49.227132,16.575804,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28685,1701951192,49.230384,16.571568,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28686,1701951194,49.230385,16.571568,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28687,1701951196,49.230386,16.571568,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28688,1701951198,49.230387,16.571568,0.0,0.0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
nr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28690 entries, 0 to 28689
Columns: 535 entries, Timestamp to TBTS6M2
dtypes: float64(450), int64(85)
memory usage: 117.1 MB


In [42]:
nr_df.isnull().sum()

Timestamp    0
Lat          0
Lon          0
BMAXMM1      0
BMAXMM2      0
            ..
TBTS5N1      0
TBTS5N2      0
TBTS5N3      0
TBTS6M1      0
TBTS6M2      0
Length: 535, dtype: int64

In [43]:
X = nr_df.iloc[:, 3:]
y = nr_df.iloc[:, 1:3]

In [44]:
X

,BMAXMM1,BMAXMM2,BMAXMM3,BMBACM1,BMBACM2,BMBAKN1,BMBAKN2,BMBAKN3,BMBARN1,BMBARN2,...,BMZVON2,BMZVON3,TBTS1M1,TBTS3M1,TBTS3M2,TBTS5N1,TBTS5N2,TBTS5N3,TBTS6M1,TBTS6M2
0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28685,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28686,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28687,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28688,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
import sklearn
from sklearn.model_selection import train_test_split

In [46]:
# Train/test split 8/2
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=81, shuffle=True, test_size=0.2)

# Confirm how the data was split
# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)

## XGBoost

In [47]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor()

In [48]:
# Train the model using the training sets
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [49]:
xgb_model.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [50]:
#run the predictions on the testing data
y_xgb_pred_test = xgb_model.predict(X_test)

In [51]:
# Determine accuracy uisng 𝑅^2
from sklearn.metrics import r2_score

In [75]:
# R^2 score
score_xgb = r2_score(y_test, y_xgb_pred_test)

print("R^2 - {}%".format(round(score_xgb, 5) *100))

R^2 - 90.7%


In [53]:
# MSE and RMSE
from sklearn.metrics import mean_squared_error

In [54]:
mse_xgb = mean_squared_error(y_test, y_xgb_pred_test)
rmse_xgb = mean_squared_error(y_test, y_xgb_pred_test, squared = False)

print(mse_xgb)
print(rmse_xgb)

7.153926859861365e-05
0.007960616583658249


In [55]:
y_test

,Lat,Lon
4288,49.210362,16.573722
24995,49.174873,16.622313
8620,49.195741,16.651359
17072,49.200876,16.637979
3185,49.223745,16.629577
...,...,...
23231,49.193773,16.567737
24019,49.173132,16.585035
27964,49.224573,16.537072
21583,49.184119,16.595165


In [56]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5738 entries, 4288 to 6927
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Lat     5738 non-null   float64
 1   Lon     5738 non-null   float64
dtypes: float64(2)
memory usage: 134.5 KB


In [57]:
y_xgb_pred_test

array([[49.207516, 16.579437],
       [49.177643, 16.618052],
       [49.197567, 16.644068],
       ...,
       [49.22272 , 16.540287],
       [49.194263, 16.608894],
       [49.206062, 16.611765]], dtype=float32)

In [58]:
len(y_xgb_pred_test)

5738

In [59]:
y_test_arr = y_test.to_numpy()

In [60]:
y_test_arr

array([[49.210362, 16.573722],
       [49.174873, 16.622313],
       [49.195741, 16.651359],
       ...,
       [49.224573, 16.537072],
       [49.184119, 16.595165],
       [49.21014 , 16.620946]])

In [61]:
from math import acos, sin, cos, radians

geo_dist_xgb = np.empty(len(y_xgb_pred_test), dtype=float)
for i in range(0, len(y_xgb_pred_test)):
  geo_dist_xgb[i] = acos(sin(radians(y_test_arr[i, 0]))*sin(radians(y_xgb_pred_test[i, 0]))+cos(radians(y_test_arr[i, 0]))*cos(radians(y_xgb_pred_test[i, 0]))*cos(radians(y_xgb_pred_test[i, 1]) - radians(y_test_arr[i, 1]))) * 6371


In [62]:
geo_dist_xgb

array([0.52205425, 0.43682746, 0.56737002, ..., 0.31131605, 1.50596175,
       0.80648975])

In [63]:
geo_dist_xgb_m = [i * 1000 for i in geo_dist_xgb]


In [76]:
# Get average of a list
def Average(lst):
    return sum(lst) / len(lst)

average_error_xgb = Average(geo_dist_xgb_m)

# Printing average of the list
print("Average error XGB [m] =", round(average_error_xgb, 5))

Average error XGB [m] = 826.48999


In [65]:
print("Max error XGB [m] = ", np.max(geo_dist_xgb_m))
print("Min error XGB [m] = ", np.min(geo_dist_xgb_m))

Max error XGB [m] =  7058.248764080266
Min error XGB [m] =  11.258141586378287


## Random forest


In [66]:
from sklearn.ensemble import RandomForestRegressor

In [67]:
rf_model = RandomForestRegressor(n_estimators=10,random_state=10)

# Train the model using the training sets
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=10)

In [68]:
#run the predictions on the training and testing data
y_rf_predict_test = rf_model.predict(X_test)

In [69]:
score_rf = r2_score(y_test, y_rf_predict_test)

print("R^2 - {}%".format(round(score_rf, 5) *100))

R^2 - 99.23100000000001%


In [70]:
# RMSE and MSE

mse_rf = mean_squared_error(y_test, y_rf_predict_test)
rmse_rf = mean_squared_error(y_test, y_rf_predict_test, squared = False)

print(mse_rf)
print(rmse_rf)

5.238414052706595e-06
0.0022039770186406235


In [71]:
geo_dist_rf = np.empty(len(y_xgb_pred_test), dtype=float)
for i in range(0, len(y_xgb_pred_test)):
  geo_dist_rf[i] = acos(sin(radians(y_test_arr[i, 0]))*sin(radians(y_rf_predict_test[i, 0]))+cos(radians(y_test_arr[i, 0]))*cos(radians(y_rf_predict_test[i, 0]))*cos(radians(y_rf_predict_test[i, 1]) - radians(y_test_arr[i, 1]))) * 6371

In [72]:
geo_dist_rf_m = [i * 1000 for i in geo_dist_rf]
geo_dist_rf_m

[32.10020600796892,
 136.00401761525944,
 5.046773297910394,
 88.30595442871224,
 205.5148577410233,
 10.593264943798214,
 18.075640843908445,
 562.4484974931746,
 17.155730351154144,
 184.2092191454607,
 160.71741246938734,
 5.172884917034818,
 3.315947402458421,
 6.089194711188511,
 56.4685494857387,
 17.89524071204517,
 15.368966284162791,
 34.309096559568296,
 190.872510329845,
 76.8659983657331,
 280.302395636567,
 45.79818891837896,
 80.20768647111822,
 81.48192154054975,
 58.90434972460567,
 7.284699114567519,
 25.731143790797606,
 10.951764941095883,
 152.37689065372928,
 78.48410808365571,
 89.85965717207452,
 2.4901295207934564,
 122.46679195414886,
 20.26262669274422,
 9.108842715945695,
 122.34530308066306,
 82.85050828599117,
 113.43222820634864,
 93.41855218914058,
 198.7488091818269,
 162.0901157075599,
 21.777255159704502,
 13.639331494640054,
 75.23662793599615,
 271.19370824597627,
 10.70413903119151,
 946.4511011741051,
 19.411192658140656,
 332.97091104126486,
 254.

In [73]:
average_error_rf = Average(geo_dist_rf_m)

# Printing average of the list
print("Average RF error [m] =", round(average_error_rf, 5))
print("Max RF error [m] =", np.max(geo_dist_rf_m))
print("Min RF error [m] =", np.min(geo_dist_rf_m))

Average RF error [m] = 131.92023
Max RF error [m] = 7893.3956795405875
Min RF error [m] = 0.0


# Cross check

In [77]:
cross_check_df = pd.read_csv('/content/drive/MyDrive/localization/LTEM_fingerprint_DB_circle.csv', sep=',')
cross_check_df

,Timestamp,Lat,Lon,BMAXMM1,BMAXMM2,BMAXMM3,BMBACM1,BMBACM2,BMBAKN1,BMBAKN2,...,BMZVON2,BMZVON3,TBTS1M1,TBTS3M1,TBTS3M2,TBTS5N1,TBTS5N2,TBTS5N3,TBTS6M1,TBTS6M2
0,1701439521,49.227657,16.575590,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
1,1701439522,49.227656,16.575592,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
2,1701439523,49.227656,16.575593,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
3,1701439524,49.227654,16.575595,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
4,1701439525,49.227653,16.575596,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6518,1701447325,49.197592,16.608559,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
6519,1701447326,49.197588,16.608551,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
6520,1701447327,49.197582,16.608546,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0
6521,1701447328,49.197578,16.608546,0,0,0,0,0,0,0,...,0,0,0,0.0,0,0,0,0,0.0,0.0


In [78]:
X_check = cross_check_df.iloc[:, 3:]
y_check = cross_check_df.iloc[:, 1:3]

## XGboost cross check

In [79]:
# Run the predictions on the cross-check data
y_xgb_cross = xgb_model.predict(X_check)

In [80]:
# R^2 score
score_xgb_cross = r2_score(y_check, y_xgb_cross)

print("R^2 - {}%".format(round(score_xgb_cross, 5) *100))

R^2 - 73.94800000000001%


In [81]:
mse_xgb_cross = mean_squared_error(y_check, y_xgb_cross)
rmse_xgb_cross = mean_squared_error(y_check, y_xgb_cross, squared = False)

print(mse_xgb_cross)
print(rmse_xgb_cross)

0.00010480961091812181
0.010046010728451701


In [82]:
y_check

,Lat,Lon
0,49.227657,16.575590
1,49.227656,16.575592
2,49.227656,16.575593
3,49.227654,16.575595
4,49.227653,16.575596
...,...,...
6518,49.197592,16.608559
6519,49.197588,16.608551
6520,49.197582,16.608546
6521,49.197578,16.608546


In [83]:
y_check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6523 entries, 0 to 6522
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Lat     6523 non-null   float64
 1   Lon     6523 non-null   float64
dtypes: float64(2)
memory usage: 102.0 KB


In [84]:
y_xgb_cross

array([[49.224243, 16.580658],
       [49.224243, 16.580658],
       [49.224243, 16.580658],
       ...,
       [49.197567, 16.608894],
       [49.197567, 16.608894],
       [49.197567, 16.608894]], dtype=float32)

In [85]:
y_check_arr = y_check.to_numpy()

In [86]:
y_check_arr

array([[49.227657, 16.57559 ],
       [49.227656, 16.575592],
       [49.227656, 16.575593],
       ...,
       [49.197582, 16.608546],
       [49.197578, 16.608546],
       [49.197576, 16.608545]])

In [87]:
# Defining the function to calculate geographical distance from coordinates
def geo_distance(true_coord, pred_coord):
  geo_dist = np.empty(len(true_coord), dtype=float)
  for i in range(0, len(true_coord)):
    geo_dist[i] = acos(sin(radians(true_coord[i, 0]))*sin(radians(pred_coord[i, 0]))+cos(radians(true_coord[i, 0]))*cos(radians(pred_coord[i, 0]))*cos(radians(pred_coord[i, 1]) - radians(true_coord[i, 1]))) * 6371
  return geo_dist

# geo_dist_xgb_check = np.empty(len(y_check), dtype=float)
# for i in range(0, len(y_check)):
#   geo_dist_xgb_check[i] = acos(sin(radians(y_check_arr[i, 0]))*sin(radians(y_xgb_pred_test[i, 0]))+cos(radians(y_test_arr[i, 0]))*cos(radians(y_xgb_pred_test[i, 0]))*cos(radians(y_xgb_pred_test[i, 1]) - radians(y_test_arr[i, 1]))) * 6371


In [88]:
geo_dist_xgb_check = geo_distance(y_check_arr, y_xgb_cross)
geo_dist_xgb_check

array([0.52871824, 0.52853731, 0.52848677, ..., 0.02536603, 0.02534079,
       0.02540349])

In [89]:
geo_dist_xgb_check_m = [i * 1000 for i in geo_dist_xgb_check]


In [90]:
average_error_xgb_check = Average(geo_dist_xgb_check_m)

# Printing average of the list
print("Average error XGB cross check [m] =", round(average_error_xgb_check, 5))

Average error XGB cross check [m] = 947.91549


In [91]:
print("Max error XGB cross check [m] = ", np.max(geo_dist_xgb_check_m))
print("Min error XGB cross check [m] = ", np.min(geo_dist_xgb_check_m))

Max error XGB cross check [m] =  12211.050179518877
Min error XGB cross check [m] =  19.798960667946655


## RF cross check

In [92]:
#run the predictions on the training and testing data
y_rf_cross = rf_model.predict(X_check)

In [93]:
score_rf_cross = r2_score(y_check, y_rf_cross)

print("R^2 - {}%".format(round(score_rf_cross, 5) *100))

R^2 - 88.84%


In [94]:
# RMSE and MSE

mse_rf_cross = mean_squared_error(y_check, y_rf_cross)
rmse_rf_cross = mean_squared_error(y_check, y_rf_cross, squared = False)

print(mse_rf_cross)
print(rmse_rf_cross)

4.5134174595079475e-05
0.006456061641110837


In [95]:
# Distance error
geo_dist_rf_check = geo_distance(y_check_arr, y_rf_cross)

In [96]:
geo_dist_rf_check_m = [i * 1000 for i in geo_dist_rf_check]
geo_dist_rf_check_m

[50.92683696584017,
 50.7723701280616,
 50.74733474750458,
 50.48862294012898,
 50.3593950179481,
 50.180826484426916,
 50.0270288063581,
 49.97836279191417,
 50.01099227839602,
 49.963392972929626,
 50.02126345566437,
 49.97448549805982,
 49.95130562978505,
 49.928024742961725,
 49.905003892193044,
 49.776524099549405,
 49.62519659515564,
 49.474134903695614,
 49.323250103839456,
 49.19499141441456,
 49.172635467596464,
 49.150269352201214,
 49.02210963552215,
 48.91628170351027,
 48.788246259799074,
 48.57654684626358,
 48.493262438401246,
 48.38739543547761,
 48.17608597404032,
 47.96487905067385,
 47.753681659338255,
 47.56571078992789,
 47.460148501987305,
 47.377945301466035,
 47.272631012499545,
 47.190769616287824,
 47.109052943607075,
 47.132674534882575,
 47.156475417490256,
 47.05143161841082,
 47.18026429334151,
 47.20423211111849,
 47.333220119834635,
 47.48634755391787,
 47.510540359645205,
 47.664043570308216,
 47.793110645110865,
 47.71308674124352,
 47.73782542813427,


In [97]:
average_error_rf_check = Average(geo_dist_rf_check_m)

# Printing average of the list
print("Average RF error cross check [m] =", round(average_error_rf_check, 5))
print("Max RF error cross check [m] =", np.max(geo_dist_rf_check_m))
print("Min RF error cross check [m] =", np.min(geo_dist_rf_check_m))

Average RF error cross check [m] = 381.6071
Max RF error cross check [m] = 6596.3592717459815
Min RF error cross check [m] = 1.55415780870913
